In [ ]:
from classes import blockchain_v1_0_0_0 as blockchain
CONNECTION_STRING = 'https://mainnet.infura.io/v3/40816bb42ce34f808b4afe219e9d41cb'
blockchain = blockchain.blockchain()

In [ ]:
from classes import blockchain_v1_0_0_0 as blockchain

In [ ]:
blockchain = blockchain.blockchain()

In [ ]:
ETHERSCAN_API_KEY = 'EWYSGQ5PEK84PRCNSCKHHZWJIWWKYYABCE'
BSCSCAN_API_KEY = 'I6ZZ57GWAS39M85QC4AJFC43Y5DUA8IU8I'
#CONNECTION_STRING = 'https://mainnet.infura.io/v3/b8932e6edfa1498ab41ce07e5686c2b4'
CONNECTION_STRING = 'https://mainnet.infura.io/v3/40816bb42ce34f808b4afe219e9d41cb'
TESTADDRESS = '0x70e36f6bf80a52b3b46b3af8e106cc0ed743e8e4' 
_contractAddress = TESTADDRESS
_apiKey = ETHERSCAN_API_KEY
_connectionString = CONNECTION_STRING

In [ ]:
blockchain.inizializeSmartContracts(_apiKey, _connectionString)

In [ ]:
blockchain.downloadContractLogsContent(_contractAddress, _apiKey, _connectionString)

In [ ]:
from classes import bitQuery_v1_0_0_0 as bitQuery
import json
bitQuery = bitQuery.bitQuery()

In [ ]:
bitQuery.getLogs(apiKey)

In [ ]:
util = util.utilities()

In [ ]:
info = util.getFrameworkInfo()

In [ ]:
apiKey = 'BQYvYOBEZWH2ymkvKTC3tbDRgprmFeOY'

In [ ]:
info['Versioning']['version']

In [ ]:
import pathlib
from os import path, listdir
import os
import configparser
import json


# Reading Configs
config = configparser.ConfigParser()
print("{0}/config.ini".format(pathlib.Path(__file__).parent.parentòknrfabsolute()))
#config.read("{0}/config.ini".format(pathlib.Path(__file__).parent.absolute()))

In [ ]:
exec(compile(open('classes/libraries_v1_0_0_0.py').read()))

In [ ]:
from past.builtins import execfile

execfile('classes/libraries_v1_0_0_0.py')

In [ ]:
pd.DataFrame()

In [ ]:
blockchain = lib.blockchain()

In [ ]:
from graphene import ObjectType, String, Schema

class Query(ObjectType):
    # this defines a Field `hello` in our Schema with a single Argument `name`
    # note: DEFINE THE DATATYPE
    hello = String(name=String(default_value="stranger"))
    goodbye = String()

    # our Resolver method takes the GraphQL context (root, info) as well as
    # Argument (name) for the Field and returns data for the query Response
    def resolve_hello(root, info, name):
        return f'Hello {name}!'

    def resolve_goodbye(root, info):
        return 'See ya!'

schema = Schema(query=Query)

In [ ]:
from graphene import ObjectType, String, Schema

In [ ]:
query = """
        {
  ethereum {
    dexTrades(options: {limit: 100, desc: "count"}) {
      count
      protocol
    }
  }
}
"""

In [ ]:
from graphene import ObjectType, String, Schema

class Query(ObjectType):
    getDexTradeByProtocol = String(
        count = String(),
        protocol = String()
    )

    def resolve_getDexTradeByProtocol(root, info):
        query = """
                {
          ethereum {
            dexTrades(options: {limit: 100, desc: "count"}) {
              count
              protocol
            }
          }
        }
        """
        return query

In [ ]:
schema = Schema(query=Query)

In [ ]:
print(schema)

In [ ]:
query_string = '{ getDexTradeByProtocol }'
result = schema.execute(query_string)
print(result)

In [ ]:
# we can query for our field (with the default argument)
query_string = '{ hello }'
result = schema.execute(query_string)
print(result.data['hello'])
# "Hello stranger!"

# or passing the argument in the query
query_with_argument = '{ hello(name: "GraphQL") },'
result = schema.execute(query_with_argument)
print(result.data['hello'])
# "Hello GraphQL!"

In [ ]:
# we can query for our field (with the default argument)
query_string = '{ goodbye }'
resesult = schema.execute(query_with_argument)
print(result.data['goodbye'])
# "Hello GraphQL!"

In [ ]:
dir(bitQuery)

In [1]:
from classes import bitQuery_v1_0_0_0 as bitQueryClass

In [2]:
apiKey = 'BQYvYOBEZWH2ymkvKTC3tbDRgprmFeOY'

In [11]:
bitQuery = bitQueryClass.bitQuery()
query = bitQueryClass.query()

In [12]:
data = bitQuery.directQuery(query.dexTradeByProtocol(), apiKey)

In [13]:
print(data)

{'data': {'ethereum': {'dexTrades': [{'count': 54174918, 'protocol': 'Uniswap v2'}, {'count': 5911016, 'protocol': 'IDEX'}, {'count': 3733490, 'protocol': 'EtherDelta'}, {'count': 2493094, 'protocol': 'Uniswap'}, {'count': 2286946, 'protocol': 'Balancer Pool Token'}, {'count': 1760217, 'protocol': 'Zerox Exchange v2'}, {'count': 1046443, 'protocol': 'Bancor Network'}, {'count': 1023483, 'protocol': 'Kyber Network v3'}, {'count': 754057, 'protocol': 'dYdX2'}, {'count': 743531, 'protocol': 'Zerox Exchange'}, {'count': 487296, 'protocol': 'Matching Market'}, {'count': 438632, 'protocol': 'Bancor Network v2'}, {'count': 419554, 'protocol': 'Zerox Exchange v3'}, {'count': 320733, 'protocol': 'Token.Store'}, {'count': 315959, 'protocol': 'Kyber Network v4'}, {'count': 264990, 'protocol': 'Curve'}, {'count': 263540, 'protocol': 'Mooniswap'}, {'count': 188464, 'protocol': 'One Inch Liquidity Pool'}, {'count': 138383, 'protocol': 'Dodo'}, {'count': 91447, 'protocol': 'Kyber Network v2'}, {'coun

In [14]:
bitQuery.getValueFromGraphQL(data, ['count', 'protocol'])

,count,protocol
0,5911016,Uniswap v2
0,3733490,IDEX
0,2493094,EtherDelta
0,2286946,Uniswap
0,1760217,Balancer Pool Token
0,1046443,Zerox Exchange v2
0,1023483,Bancor Network
0,754057,Kyber Network v3
0,743531,dYdX2
0,487296,Zerox Exchange


In [6]:
query = """
query{
  bitcoin{
    blocks{
      count
    }
   }
}
"""

In [8]:
data = bitQuery.directQuery(query, apiKey)
print(data)
bitQuery.getValueFromGraphQL(data, ['count'])

{'data': {'bitcoin': {'blocks': [{'count': 687935}]}}}


,count


In [ ]:
import json

In [ ]:
data

In [ ]:
import re

In [ ]:
data = str(data)

In [ ]:
data

In [ ]:
for match_start in re.finditer('r(count)', data):
    print('hey')

In [ ]:
a = [1] * len([1, 2])

In [ ]:
a[0]

In [ ]:
5 % 3

In [ ]:
import pandas as pd
def getValueFromGraphQL(data, _nameList):
    result = []
    for name in _nameList:
        for match_start in re.finditer(r'({0})'.format(name), data):
            tmp = data[match_start.end()+3:len(data)-match_start.end()-3]
            firstComma = re.search(', ', tmp)
            if(firstComma != None):
                firstComma = firstComma.start()
                value = data[match_start.end()+3:match_start.end()+3+firstComma].replace('}','').replace("'","") #deleting special chars
                result.append(value)
    
    # Creating DataFrame with result
    df = pd.DataFrame(columns = cols)
    tmpDf = df
    lenghtNameList = len(_nameList)
    iterableItems = len(result) // lenghtNameList - 1
    print(iterableItems)
    for i in range(0,iterableItems):
        data = []
        # Concatenate Values
        for j in range(0, lenghtNameList):
            data.append(result[j * iterableItems + i + 1])
        print(data)
        tmpDf = pd.DataFrame([data], columns = cols)
        df = pd.concat([df, tmpDf])
    return df

In [ ]:
getValueFromGraphQL(data, ['count','protocol'])

In [ ]:
a.concat(b)

In [ ]:
b

In [ ]:
cols = ['count', 'protocol']
df = pd.DataFrame(columns = cols)
tmpDf = df

In [ ]:
for i in range(0,len(a)):
    
    tmpDf = pd.DataFrame([[a[i], b[i]]], columns = cols)
    df = pd.concat([df, tmpDf])


In [ ]:
data.to_dict()

In [ ]:
df

In [15]:
import datetime    

start = datetime.datetime(2013, 1, 1)
end = datetime.datetime(2016, 1, 27)

In [16]:
start

datetime.datetime(2013, 1, 1, 0, 0)